In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

import os
dir_Alluvial = os.path.join('/content/drive/MyDrive/soil_test_correct/static/images/Dataset/Train/Alluvial soil')
dir_Black = os.path.join('/content/drive/MyDrive/soil_test_correct/static/images/Dataset/Train/Black Soil')
dir_Clay = os.path.join('/content/drive/MyDrive/soil_test_correct/static/images/Dataset/Train/Clay soil')
dir_Red = os.path.join('/content/drive/MyDrive/soil_test_correct/static/images/Dataset/Train/Red soil')

In [3]:
import tensorflow as tf
from tensorflow import keras

In [4]:
image_size = 220
batch_size = 10


target_size = (image_size, image_size)
input_shape = (image_size, image_size, 3)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)


train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/soil_test_correct/static/images/Dataset/Train',
        target_size=(200, 200),
        batch_size = batch_size,
        classes = [ 'Alluvial soil','Black Soil', 'Clay soil','Red soil',],
       class_mode='categorical')

Found 1214 images belonging to 4 classes.


In [6]:
for image_batch, label_batch in train_generator:
  break
  image_batch.shape, label_batch.shape

In [7]:
y_pred=train_generator.class_indices
print (y_pred)

{'Alluvial soil': 0, 'Black Soil': 1, 'Clay soil': 2, 'Red soil': 3}


In [8]:
model = tf.keras.models.Sequential([

    # The first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(220, 220, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # Flatten the results to feed into a dense layer
    tf.keras.layers.Flatten(),
    # 128 neuron in the fully-connected layer
    tf.keras.layers.Dense(128, activation='relu'),
    # 5 output neurons for 4 classes with the softmax activation
    tf.keras.layers.Dense(4, activation='softmax')
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 218, 218, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 109, 109, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 107, 107, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 53, 53, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 51, 51, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 25, 25, 64)        0

In [10]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['acc'])

In [12]:
total_sample = train_generator.samples  # Use the 'samples' attribute to get the total number of samples in the generator
n_epochs = 30

history = model.fit(
    train_generator,
    steps_per_epoch=total_sample // batch_size,
    epochs=n_epochs,
    verbose=1
)

Epoch 1/30
121/121 [==============================] - 64s 524ms/step - loss: 0.0428 - acc: 0.9917
Epoch 2/30
121/121 [==============================] - 55s 451ms/step - loss: 0.0533 - acc: 0.9909
Epoch 3/30
121/121 [==============================] - 51s 423ms/step - loss: 0.0306 - acc: 0.9925
Epoch 4/30
121/121 [==============================] - 52s 429ms/step - loss: 0.1038 - acc: 0.9859
Epoch 5/30
121/121 [==============================] - 54s 449ms/step - loss: 0.0705 - acc: 0.9909
Epoch 6/30
121/121 [==============================] - 53s 442ms/step - loss: 0.0613 - acc: 0.9892
Epoch 7/30
121/121 [==============================] - 53s 435ms/step - loss: 0.0732 - acc: 0.9925
Epoch 8/30
121/121 [==============================] - 54s 440ms/step - loss: 0.0590 - acc: 0.9884
Epoch 9/30
121/121 [==============================] - 53s 437ms/step - loss: 0.0818 - acc: 0.9826
Epoch 10/30
121/121 [==============================] - 53s 434ms/step - loss: 0.0344 - acc: 0.9925
Epoch 11/30
121/121

In [13]:
model.save('/content/drive/MyDrive/soil_test_correct/soil_analysis3.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
